<a href="https://colab.research.google.com/github/raz0208/Advance-Quantitative-Research-Project/blob/main/Task2/Natural_Gas_Storage_Contract_Pricing_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Natural Gas Storage Contract Pricing Prototype

In [18]:
# Import Required Libraries and Pakages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

import warnings
warnings.filterwarnings('ignore')

In [31]:
# Load and Read datasets

# Read raw dataset
Nat_Gas_df = pd.read_csv('Nat_Gas.csv')

# Check and convert data column to date format
Nat_Gas_df['Dates'] = pd.to_datetime(Nat_Gas_df['Dates'])
Nat_Gas_df = Nat_Gas_df.set_index('Dates')

print(Nat_Gas_df.head(10))

# Read ETS forecasted price dataset
ETS_Forecasted_df = pd.read_csv('/content/ETS_Nat_Gas_Forcasted_Prices(Monthly).csv')
ETS_Forecasted_df['Dates'] = pd.to_datetime(ETS_Forecasted_df['Dates'])
ETS_df = ETS_Forecasted_df.set_index('Dates')

print(ETS_Forecasted_df)

            Prices
Dates             
2020-10-31   10.10
2020-11-30   10.30
2020-12-31   11.00
2021-01-31   10.90
2021-02-28   10.90
2021-03-31   10.90
2021-04-30   10.40
2021-05-31    9.84
2021-06-30   10.00
2021-07-31   10.10
        Dates     Prices
0  2024-10-31  12.206504
1  2024-11-30  12.724507
2  2024-12-31  13.056321
3  2025-01-31  13.158380
4  2025-02-28  12.929262
5  2025-03-31  13.169657
6  2025-04-30  12.591878
7  2025-05-31  12.114226
8  2025-06-30  12.022451
9  2025-07-31  12.278492
10 2025-08-31  12.105090
11 2025-09-30  12.447106


In [23]:
# Build daily forward curve with linear interpolation + 1 year extrapolation
daily_idx = pd.date_range(df.index.min(), df.index.max() + pd.DateOffset(years=1), freq='D')
forward_curve = df.reindex(daily_idx).interpolate()

forward_curve_df = forward_curve.rename(columns={'Prices':'ForwardPrice'})
forward_curve_df

,ForwardPrice
2020-10-31,10.100000
2020-11-01,10.106667
2020-11-02,10.113333
2020-11-03,10.120000
2020-11-04,10.126667
...,...
2025-09-26,11.800000
2025-09-27,11.800000
2025-09-28,11.800000
2025-09-29,11.800000
